这是一个使用CodeFuse-Query分析JavaScript项目的教程。在教程中，你将体验到使用命令行工具对代码仓库进行数据化，然后使用Godel语言来分析这个仓库。

检查cli是否就绪

In [1]:
!which sparrow

/sparrow-cli/sparrow



STEP 0: 克隆要分析的仓库。我们以 [axios](https://github.com/axios/axios.git) 项目为例。

In [2]:
!git clone https://github.com/axios/axios.git

Cloning into 'axios'...




remote: Enumerating objects: 9324, done.        


remote: Counting objects:   0% (1/2814)        

remote: Counting objects:   1% (29/2814)        

remote: Counting objects:   2% (57/2814)        

remote: Counting objects:   3% (85/2814)        

remote: Counting objects:   4% (113/2814)        

remote: Counting objects:   5% (141/2814)        

remote: Counting objects:   6% (169/2814)        

remote: Counting objects:   7% (197/2814)        

remote: Counting objects:   8% (226/2814)        

remote: Counting objects:   9% (254/2814)        

remote: Counting objects:  10% (282/2814)        

remote: Counting objects:  11% (310/2814)        

remote: Counting objects:  12% (338/2814)        

remote: Counting objects:  13% (366/2814)        

remote: Counting objects:  14% (394/2814)        

remote: Counting objects:  15% (423/2814)        

remote: Counting objects:  16% (451/2814)        

remote: Counting objects:  17% (479/2814)        

remote: Counting objects:  18% (50

STEP 1: 代码数据化。使用 `sparrow database create` 命令创建一个db文件，指定待分析的仓库地址（当前目录下的axios子目录），分析的语言（javscript），以及db文件的存储路径（放置在当前目录下的/db/axios）。执行该命令之后，会生成一份db文件，该文件存储着代码仓库的结构化数据，之后的分析就是针对这份数据进行的。

In [3]:
!sparrow database create --source-root axios --data-language-type javascript --output ./db/axios --overwrite

2023-11-30 08:05:39,428 INFO: sparrow 2.0.0


 will start


2023-11-30 08:05:39,428 INFO: current memory is : 15.61 GB


2023-11-30 08:05:39,428 INFO: final -Xmx is : 14.61 GB


2023-11-30 08:05:39,428 INFO: /workspaces/CodeFuse-Query/tutorial/notebook/db/axios/coref_javascript_src.db will be create


2023-11-30 08:05:39,428 INFO: execute : /sparrow-cli/language/javascript/extractor/coref-javascript-src-extractor extract --src /workspaces/CodeFuse-Query/tutorial/notebook/axios --db /workspaces/CodeFuse-Query/tutorial/notebook/db/axios/coref_javascript_src.db


Ignore /workspaces/CodeFuse-Query/tutorial/notebook/axios/.git


Ignore /workspaces/CodeFuse-Query/tutorial/notebook/axios/dist


Ignore /workspaces/CodeFuse-Query/tutorial/notebook/axios/package-lock.json


Extracting /workspaces/CodeFuse-Query/tutorial/notebook/axios/.eslintrc.cjs...


Finished /workspaces/CodeFuse-Query/tutorial/notebook/axios/.eslintrc.cjs


Extracting /workspaces/CodeFuse-Query/tutorial/notebook/axios/gulpfi



Finished /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/cookies.js


Extracting /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/deprecatedMethod.js...


Finished /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/deprecatedMethod.js


Extracting /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/formDataToJSON.js...


Finished /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/formDataToJSON.js


Extracting /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/formDataToStream.js...


Finished /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/formDataToStream.js


Extracting /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/fromDataURI.js...


Finished /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/fromDataURI.js


Extracting /workspaces/CodeFuse-Query/tutorial/notebook/axios/lib/helpers/isAbsoluteURL.js...


Finished /workspaces/CodeFuse-Query/tutorial/notebook/axios

STEP 2: 使用Godel分析语言分析db文件。在本教程中，可以点击代码左侧的执行按钮，或使用快捷键：`Shift+Enter`，直接运行分析脚本。这里使用 `%db /path/to/db` 魔法命令来设置COREF db路径，内核会读取这个值来进行query查询

**示例** 查询 [axios](https://github.com/axios/axios.git) 函数级别的变更影响分析

In [4]:
%db ./db/axios
// script
use coref::javascript::*

fn default_db() -> JavascriptDB {
    return JavascriptDB::load("coref_javascript_src.db")
}

fn getACallerFunction(function: FunctionLikeDeclaration, callerFunction: FunctionLikeDeclaration) -> bool {
    for (mayInvokeExpression in MayInvokeExpression(default_db())) {
        if (mayInvokeExpression in function.getACallSite() &&
            callerFunction = mayInvokeExpression.getEnclosingFunction()) {
            return true
        }
    }
}

fn getAnEffectedFunction(function: FunctionLikeDeclaration, effectedFunction: FunctionLikeDeclaration) -> bool {
    if (getACallerFunction(function, effectedFunction)) {
        return true
    }
    for (callerFunction in FunctionLikeDeclaration(default_db())) {
        if (getACallerFunction(function, callerFunction) &&
            getAnEffectedFunction(callerFunction, effectedFunction)) {
            return true
        }
    }
}

/**
 * Query the effected functions according to the changed lines.
 *
 * @param function              the changed function id
 * @param signature             the changed function signature
 * @param functionPath          the changed function file path
 * @param startLine             the changed function start line
 * @param endLine               the changed function end line
 * @param effectedFunction      the effected function id
 * @param effectedSignature     the effected function signature
 * @param effectedFunctionPath  the effected function file path
 * @param effectedStartLine     the effected function start line
 * @param effectedEndLine       the effected function end line
 */
fn out(
    function: FunctionLikeDeclaration,
    signature: string,
    functionPath: string,
    startLine: int,
    endLine: int,
    effectedFunction: FunctionLikeDeclaration,
    effectedSignature: string,
    effectedFunctionPath: string,
    effectedStartLine: int,
    effectedEndLine: int
) -> bool {
    if (getAnEffectedFunction(function, effectedFunction)) {
        let (symbol = function.getSymbol(),
            effectedSymbol = effectedFunction.getSymbol(),
            location = function.getLocation(),
            effectedLocation = effectedFunction.getLocation()) {
            if (signature = symbol.getDescription() &&
                effectedSignature = effectedSymbol.getDescription() &&
                functionPath = location.getRelativePath() &&
                startLine = location.getStartLineNumber() &&
                endLine = location.getEndLineNumber() &&
                effectedFunctionPath = effectedLocation.getRelativePath() &&
                effectedStartLine = effectedLocation.getStartLineNumber() &&
                effectedEndLine = effectedLocation.getEndLineNumber()) {
                return true
            }
        }
    }
}

fn main() {
    output(out())
}

/workspaces/CodeFuse-Query/tutorial/notebook/db/axios


Sparrow database is set to: /workspaces/CodeFuse-Query/tutorial/notebook/db/axios


2023-11-30 08:06:18,884 INFO: sparrow 2.0.0
 will start
2023-11-30 08:06:18,884 INFO: database /workspaces/CodeFuse-Query/tutorial/notebook/db/axios/coref_javascript_src.db size: 20.82 MB
2023-11-30 08:06:18,885 INFO: execute : /sparrow-cli/godel-script/usr/bin/godel /tmp/godel-jupyter-5f14djn0/query.gdl -p /sparrow-cli/lib-1.0 -o /tmp/tmpytticwq5.gdl --verbose
[-][0.318455 ms] package scan
packages:
coref::cfamily    -> /sparrow-cli/lib-1.0/coref.cfamily.gdl
coref::go         -> /sparrow-cli/lib-1.0/coref.go.gdl
coref::java       -> /sparrow-cli/lib-1.0/coref.java.gdl
coref::javascript -> /sparrow-cli/lib-1.0/coref.javascript.gdl
coref::properties -> /sparrow-cli/lib-1.0/coref.properties.gdl
coref::python     -> /sparrow-cli/lib-1.0/coref.python.gdl
coref::sql        -> /sparrow-cli/lib-1.0/coref.sql.gdl
coref::xml        -> /sparrow-cli/lib-1.0/coref.xml.gdl
modules
+--coref -> coref
|--cfamily -> coref::cfamily
|--go -> coref::go
|--properties -> coref::properties
|--java -> coref::

,function,signature,functionPath,startLine,endLine,effectedFunction,effectedSignature,effectedFunctionPath,effectedStartLine,effectedEndLine
0,-9156169672968821161,(method) AxiosHeaders.from(thing: any): AxiosH...,lib/core/AxiosHeaders.js,248,250,-8249170294854635473,(local function) onAdapterRejection(reason: an...,lib/core/dispatchRequest.js,64,80
1,-9156169672968821161,(method) AxiosHeaders.from(thing: any): AxiosH...,lib/core/AxiosHeaders.js,248,250,1405045491285225472,(local function) onloadend(): void,lib/adapters/xhr.js,92,121
2,-9156169672968821161,(method) AxiosHeaders.from(thing: any): AxiosH...,lib/core/AxiosHeaders.js,248,250,1808319753094699064,(local function) dispatchHttpRequest(resolve: ...,lib/adapters/http.js,161,678
3,-9156169672968821161,(method) AxiosHeaders.from(thing: any): AxiosH...,lib/core/AxiosHeaders.js,248,250,2322743179144353808,(local function) onAdapterResolution(response:...,lib/core/dispatchRequest.js,51,64
4,-9156169672968821161,(method) AxiosHeaders.from(thing: any): AxiosH...,lib/core/AxiosHeaders.js,248,250,3763705685345704826,"function transformData(fns: Array | {}, respon...",lib/core/transformData.js,15,28
...,...,...,...,...,...,...,...,...,...,...
924,9099947820526391919,"function toFormData(obj: any, formData: string...",lib/helpers/toFormData.js,86,217,8914351007635312770,(local function) dispatchXhrRequest(resolve: a...,lib/adapters/xhr.js,49,259
925,9099947820526391919,"function toFormData(obj: any, formData: string...",lib/helpers/toFormData.js,86,217,9177673545987323723,(method) Axios.getUri(config: any): string,lib/core/Axios.js,161,165
926,9144720937035304957,function generateReadableStream(length?: numbe...,test/unit/adapters/http.js,126,144,278810980780541004,(local function)(): any,test/unit/adapters/http.js,864,889
927,9144720937035304957,function generateReadableStream(length?: numbe...,test/unit/adapters/http.js,126,144,9144720937035304957,function generateReadableStream(length?: numbe...,test/unit/adapters/http.js,126,144


保存上一次运行的 query 结果保存到一个JSON文件

In [5]:
%%save_to ./query.json

Query result saved to /workspaces/CodeFuse-Query/tutorial/notebook/query.json


STEP 3: 现在可以针对分析生成的结果，进行进一步的代码分析了，假设我们要分析`'lib/core/AxiosHeaders.js'`文件的变更有可能会影响到哪一些函数，我们可以这么进行分析：

In [8]:
%%python
import json

with open('./query.json', 'r') as fp:
    query_data = json.loads(fp.read())

analysis_path = 'lib/core/AxiosHeaders.js'
print(f'Modification of file {analysis_path} for the following function might have impacts:')
for data in query_data:
    if data['functionPath'] == analysis_path:
        signature = data['signature']
        functionPath = data['functionPath']
        effectedSignature = data['effectedSignature']
        effectedFunctionPath = data['effectedFunctionPath']
        effectedStartLine = data['effectedStartLine']
        print(f'  --> `{signature}` might impact `{effectedSignature}` in {effectedFunctionPath} at line {effectedStartLine}')

Modification of file lib/core/AxiosHeaders.js for the following function might have impacts:
  --> `(method) AxiosHeaders.from(thing: any): AxiosHeaders` might impact `(local function) onAdapterRejection(reason: any): any` in lib/core/dispatchRequest.js at line 64
  --> `(method) AxiosHeaders.from(thing: any): AxiosHeaders` might impact `(local function) onloadend(): void` in lib/adapters/xhr.js at line 92
  --> `(method) AxiosHeaders.from(thing: any): AxiosHeaders` might impact `(local function) dispatchHttpRequest(resolve: any, reject: any, onDone: any): unknown` in lib/adapters/http.js at line 161
  --> `(method) AxiosHeaders.from(thing: any): AxiosHeaders` might impact `(local function) onAdapterResolution(response: any): any` in lib/core/dispatchRequest.js at line 51
  --> `(method) AxiosHeaders.from(thing: any): AxiosHeaders` might impact `function transformData(fns: Array | {}, response: any | null): any` in lib/core/transformData.js at line 15
  --> `(method) AxiosHeaders.from(